In [1]:
import re
import os
import json

### Clubbing *INV or *PAR, %wor, %mor, %gra

- from each file, pull out a set of lines that have INV/PAR, wor, mor, gra and club them into one array of 4 sentences along with timecode for the whole sentence
- append this list into a file list for that particular corpus
- this file list will be appended to the global list of lists with content from all the files 

In [2]:
def Individual_Club_Structure(club, speaker):
    ind_club = {}
    ind_club["speaker"] = speaker

    chunk = ""
    club_line_num = 0
    for club_line in club.splitlines():
        if club_line.startswith(" %mor"):
            ind_club["dialogue"] = chunk
            chunk = ""
            chunk += club_line + "\n" + " "
            club_line_num = club.splitlines().index(club_line)
            break
        else:
            chunk += club_line + "\n" + " "
    
    for club_line in club.splitlines()[club_line_num:]:
        if club_line.startswith(" %gra"):
            ind_club["mor"] = chunk
            chunk = ""
            chunk += club_line + "\n" + " "
            club_line_num = club.splitlines().index(club_line)
            break
        else:
            chunk += club_line + "\n" + " "
    
    for club_line in club.splitlines()[club_line_num:]:
        if club_line.startswith(" %wor"):
            ind_club["gra"] = chunk
            chunk = ""
            club_line_num = club.splitlines().index(club_line)
            break
        else:
            chunk += club_line + "\n" + " "
    
    for club_line in club.splitlines()[club_line_num:]:
        # iterate till end and add to %wor 
        chunk += club_line + "\n" + " "

        
    ind_club["wor"] = chunk

    return ind_club



def Club(file):
    ''' # structure of file dict: 
        # {
        # "metadata": "<string>", 
        # "clubs":  [
        #               {
        #                   
                            #   "speaker" : "<speaker_type>",
                            #   "dialogue": "<dialogue text>",
                            #   "mor": "<morphology>",
                            #   "gra": "<grammar>",
                            #   "wor": "<word>"
                            
        #               }, 
        #               { }, ...
        #           ]
        # } '''
   
    file_dict = {}

    with open(file, 'r') as f:
        # store whole_text 
        whole_text = f.read()
        
        club = ""
        # first iterate through all meta data until we hit @G
        break_line_num= 0
        for line in whole_text.splitlines():
            if line.startswith("@G"):
                club += line + "\n" + " "
                break_line_num = whole_text.splitlines().index(line)
                break
            else:
                club += line + "\n" + " "
        
        # store meta data in file_list under "metadata"
        file_dict["metadata"] = club
        # print("metadata: ", file_dict["metadata"])

        # iterate through all clubs from break_line_num to end using:
            # use a for loop to add lines to club from *PAR until we hit *INV or *PAR
            # *INV , *PAR mark the begining of a dialogue
            # %wor, %mor, gra -> breaks down the sentence and list its qualities like morphology and grammar
            # keep adding strings to club until we hit a new dialogue represented by *PAR or *INV
        file_dict["clubs"] = []
        club = ""
        i=0
        speaker = ""
        for line in whole_text.splitlines()[break_line_num+1:]:
            # print(line)
            if (line.startswith("*INV") or line.startswith("*PAR")) and club == "":
                if line.startswith("*INV"):
                    speaker = "INV"
                else:
                    speaker = "PAR"

            if (line.startswith("*INV") or line.startswith("*PAR")) and club!="":
                # iterate the club and chunk it based on dialogues begining with:
                    # *INV or *PAR
                    # *mor
                    # *gra
                    # *wor
                
                # iterate through club and chunk it based on dialogues
                # speaker type identified by checking if it starts with *INV or *PAR in the above if check
                # dialogue = chunk that is between *INV to %mor 
                # mor = chunk that is between %mor to %gra
                # gra = chunk that is between %gra to %wor
               
                ind_club = Individual_Club_Structure(club,speaker)
                file_dict["clubs"].append(ind_club)
    
                club = ""
                club += line + "\n" + " "
                if line.startswith("*INV"):
                    speaker = "INV"
                else:
                    speaker = "PAR"
            else:
                club += line + "\n" + " "
        
        # add last club to file_list
        file_dict["clubs"].append(Individual_Club_Structure(club, speaker))

    return file_dict

file_list = Club('./English_Protocol_Aphasia/Garrett/garrett01a.cha')

# print file list in a formatted way
# print(json.dumps(file_list, indent=4))

    

In [3]:
# now use the above functions to store all files in the directory:

# dir_dict = {
#        "metadata": "<text from metadata file>",
#        "files": [
#            {
#                "filename": "<filename>",
#                "file": "<file dict>"
#            },
#            {
#                "filename": "<filename>",
#                "file": "<file dict>"
#            },
#            ...
#        ]
#    }

def Directory(dir):
    dir_dict = {}
    # iterate through all files in the directory
    # store all files in a list
    # store all file names in a list
    # store all file dicts in a list
    # store
    file_list = []
    file_name_list = []
    file_dict_list = []
    for file in os.listdir(dir):
        if file.endswith(".cha"):
            file_list.append(file)
            file_name_list.append(file)
            file_dict_list.append(Club(dir + "/" + file))
        
        if(file.endswith(".cdc")):
            with open(dir + "/" + file, 'r') as f:
                dir_dict["metadata"] = f.read()
        
    dir_dict["files"] = []
    for i in range(len(file_list)):
        dir_dict["files"].append({"filename": file_name_list[i], "file": file_dict_list[i]})
    
    return dir_dict

dir_dict = Directory('./English_Protocol_Aphasia/Garrett')

# print dir_dict in a formatted way
# print(json.dumps(dir_dict, indent=4))



# now use the above functions to store all directories in the directory_list:
Global_Dir_list = []
for dir in os.listdir('./English_Protocol_Aphasia'):
    Global_Dir_list.append(Directory('./English_Protocol_Aphasia/' + dir))

# print Global_Dir_list in a formatted way
print(json.dumps(Global_Dir_list, indent=4))

## Pauses

   
1. "+..?" or "+..." -> replace with @tpause@ | Trailing off  
2. "(.)" -> @pause@
3. "(..) -> @lpause@
4. "(...) -> @xlpause@


In [ ]:
def 

## Interruptions 
1. "+/." or "+/?" -> replace the next dialogue and this encoding with @ipause@ and join the two dialogues of the same speaker | Interruption
2. "+//." or "+//?" -> remove this encoding and join the current and next dialogue